In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
dbscan_clusters = pd.read_csv('../../data/evaluation_data/dbscan_clusters.csv', index_col=0)
average_house_price = pd.read_csv('../../data/evaluation_data/average_house_price.csv')
median_house_price = pd.read_csv('../../data/evaluation_data/median_house_price.csv')

In [3]:
# clean average house price

for column in average_house_price.loc[:,'studio':]:
    average_house_price[column] = average_house_price[column].str.replace(',','')
    average_house_price[column] = average_house_price[column].str.replace('$','')
    average_house_price[column] = average_house_price[column].str.replace(' ','')
    average_house_price[column] = average_house_price[column].str.replace('-','0')
    average_house_price[column] = average_house_price[column].astype(int)

/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_19615/1913209532.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  average_house_price[column] = average_house_price[column].str.replace('$','')


In [4]:
# add mean of house types column
average_house_price['average_price'] = average_house_price.loc[:,'studio':].mean(axis=1)

In [5]:
# drop other columns
average_house_price = average_house_price.drop(['studio', '1_bedroom', '2_bedroom', '3_bedroom'], axis=1)

In [6]:
median_house_price = median_house_price.loc[:,:'type']

In [7]:
median_house_price['average price']

0      $1,975
1      $1,650
2      $1,731
3      $1,898
4      $2,491
        ...  
531    $1,983
532    $1,400
533    $5,345
534    $1,800
535    $1,377
Name: average price, Length: 536, dtype: object

In [8]:
# clean median prices

for value in median_house_price:
    print(value)
    median_house_price[value] = median_house_price[value].str.replace(',','')
    median_house_price[value] = median_house_price[value].str.replace('$','')
    median_house_price[value] = median_house_price[value].str.replace(' ','')
    median_house_price[value] = median_house_price[value].str.replace('-','0')

neighborhood
average price
type


/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_19615/4237213198.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  median_house_price[value] = median_house_price[value].str.replace('$','')


In [9]:
median_house_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   neighborhood   536 non-null    object
 1   average price  536 non-null    object
 2   type           536 non-null    object
dtypes: object(3)
memory usage: 12.7+ KB


In [10]:
median_house_price['average price'] = median_house_price['average price'].astype(int)

In [11]:
median_house_price = median_house_price.groupby('neighborhood').mean().reset_index()

In [12]:
median_house_price = pd.DataFrame(median_house_price)

In [13]:
median_house_price

,neighborhood,average price
0,Allerton,1625.000000
1,Annadale,1450.000000
2,ArdenHeights,2250.000000
3,Astoria,1883.333333
4,Auburndale,1916.666667
...,...,...
211,Woodhaven,1702.333333
212,WoodlawnHeights,1900.000000
213,Woodrow,1400.000000
214,Woodside,1807.666667


In [14]:
average_house_price

,borough,neighborhood,average_price
0,manhatan,Chelsea,6221.00
1,manhatan,East Village,3805.25
2,manhatan,Flatiron/Union Square,8128.25
3,manhatan,Gramercy Park,4716.00
4,manhatan,Greenwich Village,6840.50
5,manhatan,Battery Park City,5957.25
6,manhatan,Financial District,5909.50
7,manhatan,Lower East Side,4119.00
8,manhatan,NoHo,9332.25
9,manhatan,NoLiTa/Little Italy,5611.00


In [15]:
# remove borough
average_house_price = average_house_price.drop('borough', axis=1)

In [16]:
# let's merge the two
house_costs = pd.concat([average_house_price.set_index('neighborhood'), median_house_price.set_index('neighborhood')], join='outer', axis=1).reset_index()

In [49]:
# stack on neighborhood
house_costs

,neighborhood,average_price
0,Chelsea,5712
1,East Village,3805
2,Flatiron/Union Square,8128
3,Gramercy Park,4716
4,Greenwich Village,6841
...,...,...
256,Woodhaven,1702
257,WoodlawnHeights,1900
258,Woodrow,1400
259,Woodside,1808


In [50]:
# There are 160 missing values - :( solution could be to geocode
missing = []
for hood in list(house_costs['neighborhood']):
    if hood not in list(dbscan_clusters['neighborhood']):
        missing.append(hood)
missing

['Flatiron/Union Square',
 'Gramercy Park',
 'NoHo',
 'NoLiTa/Little Italy',
 'SoHo',
 'Beekman/Sutton Place',
 'Midtown East',
 'Midtown West',
 'Turtle Bay/United Nations',
 'Park/Fifth Ave. to 79th St.',
 'Broadway Corridor',
 'Central Park West',
 'Lincoln Center',
 'Riverside Dr./West End Ave.',
 'Harlem',
 'Bedford-Stuyvesant',
 'Downtown Brooklyn',
 'DUMBO',
 'South Slope - Greenwood Heights',
 'ArdenHeights',
 'BathBeach',
 'BatteryParkCity',
 'BayRidge',
 'Bedford0Stuyvesant',
 'BedfordPark',
 'BergenBeach',
 'BoerumHill',
 'BoroughPark',
 'Bowery',
 'BrightonBeach',
 'BronxPark',
 'BrooklynHeights',
 'BrooklynNavyYard',
 'BullsHead',
 'CarrollGardens',
 'CastleHill',
 'CastletonCorners',
 'CentralHarlem',
 'CentralPark',
 'CityIsland',
 'CivicCenter',
 'ClasonPoint',
 'Clearview',
 'ClintonHill',
 'Co0OpCity',
 'CobbleHill',
 'CollegePoint',
 'ConcourseVillage',
 'ConeyIsland',
 'CountryClub',
 'CrotonaParkEast',
 'CrownHeights',
 'CypressHills',
 'DonganHills',
 'DowntownBro

In [56]:
import geocoder

cluster_hoods = list(dbscan_clusters['neighborhood'])
price_hoods = list(house_costs['neighborhood'])

In [ ]:
    gplace_params = {
        "key": google_places_key,
        "location": latitude+','+longitude,
        "radius": radius,
        "keyword": query
    }

In [ ]:
os.environ["GOOGLE_API_KEY"]

In [62]:
price_hoods[0]

'Chelsea'